In [4]:
import wandb

# wandb.init(project="VAE", entity="graphs-stats-and-ml")

wandb: Currently logged in as: graphs-stats-and-ml (use `wandb login --relogin` to force relogin)


In [1]:
from model2 import TOCCA1
from model2 import TOCCA
from model2 import TOCCA3
from model2 import TOCCA4
from model2 import TOCCA5
from aug import random_aug
from dataset import load

import numpy as np
import torch as th
import torch.nn as nn

import warnings

warnings.filterwarnings('ignore')

Using backend: pytorch


In [2]:
# parser.add_argument('--gpu', type=int, default=0, help='GPU index.')
# parser.add_argument('--use_mlp', action='store_true', default=False, help='Use MLP instead of GNN')

In [2]:
graph, feat, labels, num_class, train_idx, val_idx, test_idx = load('pubmed')
graph = graph.remove_self_loop().add_self_loop()

in_dim = feat.shape[1]

hid_dim = 512
out_dim = 512
n_layers = 2
n_classes = 7
loss_fn = nn.CrossEntropyLoss()

label = labels
train_labels = label[train_idx]
val_labels = label[val_idx]
test_labels = label[test_idx]
    
best_val_acc = 0
eval_acc = 0

#best_val_acc2 = 0 
#eval_acc2 = 0 

model = TOCCA(in_dim, hid_dim, out_dim, n_classes, n_layers, use_mlp=False)
lr1 = 1e-3
wd1 = 0
wd2 = 1e-4
optimizer = th.optim.Adam(model.parameters(), lr=lr1, weight_decay=wd2)

N = graph.number_of_nodes()


for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    dfr = 0.2
    der = 0.2

    graph1, feat1 = random_aug(graph, feat, 0.2, 0.2)
    graph2, feat2 = random_aug(graph, feat, 0.2, 0.2)

    graph1 = graph1.add_self_loop()
    graph2 = graph2.add_self_loop()

    z1, z2, logits = model(graph1, feat1, graph2, feat2)
    # z1, z2, logits, logits2 = model(graph1, feat1, graph2, feat2)

    c = th.mm(z1.T, z2)
    c1 = th.mm(z1.T, z1)
    c2 = th.mm(z2.T, z2)

    c = c / N
    c1 = c1 / N
    c2 = c2 / N

    loss_inv = -th.diagonal(c).sum()
    iden = th.tensor(np.eye(c.shape[0]))
    loss_dec1 = (iden - c1).pow(2).sum()
    loss_dec2 = (iden - c2).pow(2).sum()

    lambd1 = 0 # best 1e-2
    lambd2 = 1e-4

    train_embs = logits[train_idx]
    val_embs = logits[val_idx]
    test_embs = logits[test_idx]

    # train_embs2 = logits2[train_idx] #
    # val_embs2 = logits2[val_idx] #
    # test_embs2 = logits2[test_idx] #

    preds = th.argmax(train_embs, dim=1)
    # preds2 = th.argmax(train_embs2, dim=1) #

    train_acc = th.sum(preds == train_labels).float() / train_labels.shape[0]
    # train_acc2 = th.sum(preds2 == train_labels).float() / train_labels.shape[0] #
    loss_task = loss_fn(train_embs, train_labels)
    # loss_task2 = loss_fn(train_embs2, train_labels) #

    loss = lambd1 * loss_task + loss_inv + lambd2 * (loss_dec1 + loss_dec2) # 

    loss.backward()
    optimizer.step()

    model.eval()
    with th.no_grad():
        val_preds = th.argmax(val_embs, dim=1)
        test_preds = th.argmax(test_embs, dim=1)
        # val_preds2 = th.argmax(val_embs2, dim=1)
        # test_preds2 = th.argmax(test_embs2, dim=1)

        val_acc = th.sum(val_preds == val_labels).float() / val_labels.shape[0]
        test_acc = th.sum(test_preds == test_labels).float() / test_labels.shape[0]
        #val_acc2 = th.sum(val_preds2 == val_labels).float() / val_labels.shape[0]
        #test_acc2 = th.sum(test_preds2 == test_labels).float() / test_labels.shape[0]

        if val_acc >= best_val_acc:
            best_val_acc = val_acc
            if test_acc > eval_acc:
                eval_acc = test_acc

    print('Epoch:{}, loss={:.4f}, train_acc:{:.4f}, val_acc:{:4f}, test_acc:{:4f}'.format(epoch, loss.item(), train_acc, val_acc, test_acc))
    print('test accuracy:{:.4f}'.format(eval_acc))

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch:0, loss=-360.9024, train_acc:0.1667, val_acc:0.180000, test_acc:0.174000
test accuracy:0.1740
Epoch:1, loss=-378.2430, train_acc:0.1167, val_acc:0.148000, test_acc:0.169000
test accuracy:0.1740
Epoch:2, loss=-384.7776, train_acc:0.1333, val_acc:0.150000, test_acc:0.165000
test accuracy:0.1740
Epoch:3, loss=-392.8266, train_acc:0.1167, val_acc:0.154000, test_acc:0.166000
test accuracy:0.1740
Epoch:4, loss=-395.2006, train_acc:0.1500, val_acc:0.148000, test_acc:0.181000
test accuracy:0.1740
Epoch:5, loss=-429.2599, train_acc:0.1333, val_acc:0.160000, test_acc:0.184000
test accuracy:0.1740
Epoch:6, loss=-433.9079, train_acc:0.1000, val_acc:0.148000, test_acc:0.167000
test accuracy:0.1740
Epoch:7, loss=-428.8533, train_acc:0.1333, val_acc:0.172000, test_acc:0.190000
test accuracy:0.1740
Epoch:8, loss=-426.

Epoch:80, loss=-477.0278, train_acc:0.1333, val_acc:0.054000, test_acc:0.067000
test accuracy:0.1840
Epoch:81, loss=-476.3353, train_acc:0.1000, val_acc:0.056000, test_acc:0.065000
test accuracy:0.1840
Epoch:82, loss=-476.2201, train_acc:0.1167, val_acc:0.056000, test_acc:0.074000
test accuracy:0.1840
Epoch:83, loss=-475.2142, train_acc:0.1333, val_acc:0.062000, test_acc:0.080000
test accuracy:0.1840
Epoch:84, loss=-476.8502, train_acc:0.1500, val_acc:0.048000, test_acc:0.078000
test accuracy:0.1840
Epoch:85, loss=-476.4197, train_acc:0.1500, val_acc:0.064000, test_acc:0.073000
test accuracy:0.1840
Epoch:86, loss=-478.0865, train_acc:0.1000, val_acc:0.054000, test_acc:0.073000
test accuracy:0.1840
Epoch:87, loss=-476.3641, train_acc:0.1500, val_acc:0.072000, test_acc:0.078000
test accuracy:0.1840
Epoch:88, loss=-476.8681, train_acc:0.1333, val_acc:0.074000, test_acc:0.070000
test accuracy:0.1840
Epoch:89, loss=-476.2078, train_acc:0.1000, val_acc:0.054000, test_acc:0.067000
test accura

In [5]:
# wandb.init(config={"lambd1": 0})
# wandb.log({'accuracy': eval_acc})
# wandb.config.update({"lr": 0.1, "channels": 16})

In [4]:
citeseer = [71.2, 70.8, 70.9, 71.3, 70.3]
cora = [82, 80.5, 81.4]

tensor(0.8200)

In [2]:
graph, feat, labels, num_class, train_idx, val_idx, test_idx = load('cora')
graph = graph.remove_self_loop().add_self_loop()

in_dim = feat.shape[1]

hid_dim = 512
out_dim = 512
n_layers = 2
n_classes = 7
loss_fn = nn.CrossEntropyLoss()

label = labels
train_labels = label[train_idx]
val_labels = label[val_idx]
test_labels = label[test_idx]
    
best_val_acc = 0
eval_acc = 0

#best_val_acc2 = 0 
#eval_acc2 = 0 

model = TOCCA(in_dim, hid_dim, out_dim, n_classes, n_layers, use_mlp=False)
lr1 = 1e-3
wd1 = 0
wd2 = 1e-4
optimizer = th.optim.Adam(model.parameters(), lr=lr1, weight_decay=wd2)

N = graph.number_of_nodes()

lambda2_list = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6] 
lambda2_data = th.zeros((len(lambda2_list), 5))

for j, lambda2_value in enumerate(lambda2_list):
    for n in range(5):
        for epoch in range(100):
            model.train()
            optimizer.zero_grad()

            dfr = 0.2
            der = 0.2

            graph1, feat1 = random_aug(graph, feat, 0.2, 0.2)
            graph2, feat2 = random_aug(graph, feat, 0.2, 0.2)

            graph1 = graph1.add_self_loop()
            graph2 = graph2.add_self_loop()

            z1, z2, logits = model(graph1, feat1, graph2, feat2)
            # z1, z2, logits, logits2 = model(graph1, feat1, graph2, feat2)

            c = th.mm(z1.T, z2)
            c1 = th.mm(z1.T, z1)
            c2 = th.mm(z2.T, z2)

            c = c / N
            c1 = c1 / N
            c2 = c2 / N

            loss_inv = -th.diagonal(c).sum()
            iden = th.tensor(np.eye(c.shape[0]))
            loss_dec1 = (iden - c1).pow(2).sum()
            loss_dec2 = (iden - c2).pow(2).sum()

            lambd1 = 10^2
            lambd2 = lambda2_value # 1e-5

            train_embs = logits[train_idx]
            val_embs = logits[val_idx]
            test_embs = logits[test_idx]

            # train_embs2 = logits2[train_idx] #
            # val_embs2 = logits2[val_idx] #
            # test_embs2 = logits2[test_idx] #

            preds = th.argmax(train_embs, dim=1)
            # preds2 = th.argmax(train_embs2, dim=1) #

            train_acc = th.sum(preds == train_labels).float() / train_labels.shape[0]
            # train_acc2 = th.sum(preds2 == train_labels).float() / train_labels.shape[0] #
            loss_task = loss_fn(train_embs, train_labels)
            # loss_task2 = loss_fn(train_embs2, train_labels) #

            loss = loss_task + lambd1 * loss_inv + lambd2 * (loss_dec1 + loss_dec2) # 

            loss.backward()
            optimizer.step()

            model.eval()
            with th.no_grad():
                val_preds = th.argmax(val_embs, dim=1)
                test_preds = th.argmax(test_embs, dim=1)
                # val_preds2 = th.argmax(val_embs2, dim=1)
                # test_preds2 = th.argmax(test_embs2, dim=1)

                val_acc = th.sum(val_preds == val_labels).float() / val_labels.shape[0]
                test_acc = th.sum(test_preds == test_labels).float() / test_labels.shape[0]
                #val_acc2 = th.sum(val_preds2 == val_labels).float() / val_labels.shape[0]
                #test_acc2 = th.sum(test_preds2 == test_labels).float() / test_labels.shape[0]

                if val_acc >= best_val_acc:
                    best_val_acc = val_acc
                    if test_acc > eval_acc:
                        eval_acc = test_acc

            #print('Epoch:{}, loss={:.4f}, train_acc:{:.4f}, val_acc:{:4f}, test_acc:{:4f}'.format(epoch, loss.item(), train_acc, val_acc, test_acc))
            #print('test accuracy:{:.4f}'.format(eval_acc))
        lambda2_data[j, n] = eval_acc
            

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
